# A3: A\*, IDS, and Effective Branching Factor

For this assignment, implement the Recursive Best-First Search
implementation of the A\* algorithm given in class.  Name this function `aStarSearch`.  Also in this notebook include your `iterativeDeepeningSearch` functions.  Define a new function named `ebf` that returns an estimate of the effective
branching factor for a search algorithm applied to a search problem.

So, the required functions are

   - `aStarSearch(startState, actionsF, takeActionF, goalTestF, hF)`
   - `iterativeDeepeningSearch(startState, goalState, actionsF, takeActionF, maxDepth)`
   - `ebf(nNodes, depth)`, returns number of nodes expanded and depth reached during a search.

Apply `iterativeDeepeningSearch` and `aStarSearch` to several eight-tile sliding puzzle
problems. Compare their results by displayng
solution path depth, number of nodes 
generated, and the effective branching factor, and discuss the results.  Do this by defining the following function that prints the table as shown in the example below.

   - 'runExperiment(goalState1, goalState2, goalState3, [h1, h2, h3])

## Heuristic Functions

For `aStarSearch` use the following two heuristic functions, plus one more of your own design, for a total of three heuristic functions.

  * `h1(state, goal)`: $h(state, goal) = 0$, for all states $state$ and all goal states $goal$,
  * `h2(state, goal)`: $h(state, goal) = m$, where $m$ is the Manhattan distance that the blank is from its goal position,
  * `h3(state, goal)`: $h(state, goal) = ?$, that you define.  It must be admissible, and not constant for all states.

## Comparison

Apply all four algorithms (`iterativeDeepeningSearch` plus `aStarSearch` with the three heuristic
functions) to three eight-tile puzzle problems with start state

$$
\begin{array}{ccc}
1 & 2 & 3\\
4 & 0 & 5\\
6 & 7 & 8
\end{array}
$$

and these three goal states.

$$
\begin{array}{ccccccccccc}
1 & 2 & 3  & ~~~~ & 1 & 2 & 3  &  ~~~~ & 1 & 0 &  3\\
4 & 0 & 5  & & 4 & 5 & 8  & & 4 & 5 & 8\\
6 & 7 & 8 &  & 6 & 0 & 7  & & 2 & 6 & 7
\end{array}
$$

Print a well-formatted table like the following.  Try to match this
format. 

           [1, 2, 3, 4, 0, 5, 6, 7, 8]    [1, 2, 3, 4, 5, 8, 6, 0, 7]    [1, 0, 3, 4, 5, 8, 2, 6, 7] 
    Algorithm    Depth  Nodes  EBF              Depth  Nodes  EBF              Depth  Nodes  EBF          
         IDS       0      0  0.000                3     43  3.086               11 225850  2.954         
        A*h1       0      0  0.000                3    116  4.488               11 643246  3.263         
        A*h2       0      0  0.000                3     51  3.297               11 100046  2.733         

Of course you will have one more line for `h3`.

Download [A3grader.tar](http://www.cs.colostate.edu/~anderson/cs440/notebooks/A3grader.tar) and extract A3grader.py from it.

# IDS

In [1]:
class Node:
    def __init__(self, state,f=0, g=0 ,h=0):
        self.state = state
        self.f = f
        self.g = g
        self.h = h  
        self.parent=Node
    def __repr__(self):
        return "Node(" + repr(self.state) + ", f=" + repr(self.f) + \
               ", g=" + repr(self.g) + ", h=" + repr(self.h) + ")"

In [111]:
def ebf(nodes, depth, precision=0.001):
    result=0
    d=depth
    b=0
    while (nodes-result) > precision:
        result=(1-b**(d+1))/(1-b)
        b+=precision
        
    return b
ebf( 2,1)


1.0010000000000006

In [129]:
#%%writefile aStarSearch.py
# Recursive Best First Search (Figure 3.26, Russell and Norvig)
#  Recursive Iterative Deepening form of A*, where depth is replaced by f(n)

'''class Node:
    def __init__(self, state, f=0, g=0 ,h=0):
        self.state = state
        self.f = f
        self.g = g
        self.h = h
    def __repr__(self):
        return "Node(" + repr(self.state) + ", f=" + repr(self.f) + \
               ", g=" + repr(self.g) + ", h=" + repr(self.h) + ")"'''

def aStarSearch(startState, actionsF, takeActionF, goalTestF, hF):
    global nNodes
    global depth
    nNodes=0
    depth=0
    ebfValue=0
    h = hF(startState)
    startNode = Node(state=startState, f=0+h, g=0, h=h)
    a= aStarSearchHelper(startNode, actionsF, takeActionF, goalTestF, hF, float('inf'))
    depth = len(a[0])-1
    
    return a#,ebfValue

def aStarSearchHelper(parentNode, actionsF, takeActionF, goalTestF, hF, fmax):
    global nNodes
    if goalTestF(parentNode.state):
        return ([parentNode.state], parentNode.g)
    ## Construct list of children nodes with f, g, and h values
    actions = actionsF(parentNode.state)
    if not actions:
        return ("failure", float('inf'))
    children = []
    for action in actions:
        (childState,stepCost) = takeActionF(parentNode.state, action)
        h = hF(childState)
        g = parentNode.g + stepCost
        f = max(h+g, parentNode.f)
        childNode = Node(state=childState, f=f, g=g, h=h)
        children.append(childNode)
        nNodes+=1

    while True:
        # find best child
        children.sort(key = lambda n: n.f) # sort by f value
        bestChild = children[0]
        if bestChild.f > fmax:
            return ("failure",bestChild.f)
        # next lowest f value
        alternativef = children[1].f if len(children) > 1 else float('inf')
        # expand best child, reassign its f value to be returned value
        result,bestChild.f = aStarSearchHelper(bestChild, actionsF, takeActionF, goalTestF,
                                            hF, min(fmax,alternativef))
        if result is not "failure":               #        g
            result.insert(0,parentNode.state)     #       / 
            return (result, bestChild.f)  
            
                                                  #      d
                                                  #     / \ 



In [110]:
aStarSearch([1, 2, 3, 4, 0, 5, 6, 7, 8],actionsF_8p, takeActionF_8p,
                     lambda s: goalTestF_8p(s, [1, 0, 3, 4, 5, 8, 2, 6, 7]),
    lambda s: h3_8p(s,[1, 0, 3, 4, 5, 8, 2, 6, 7]))



nodes 5232
depth 11
2.051


([[1, 2, 3, 4, 0, 5, 6, 7, 8],
  [1, 0, 3, 4, 2, 5, 6, 7, 8],
  [0, 1, 3, 4, 2, 5, 6, 7, 8],
  [4, 1, 3, 0, 2, 5, 6, 7, 8],
  [4, 1, 3, 2, 0, 5, 6, 7, 8],
  [4, 1, 3, 2, 5, 0, 6, 7, 8],
  [4, 1, 3, 2, 5, 8, 6, 7, 0],
  [4, 1, 3, 2, 5, 8, 6, 0, 7],
  [4, 1, 3, 2, 5, 8, 0, 6, 7],
  [4, 1, 3, 0, 5, 8, 2, 6, 7],
  [0, 1, 3, 4, 5, 8, 2, 6, 7],
  [1, 0, 3, 4, 5, 8, 2, 6, 7]],
 11)

In [4]:

successors = {'a': ['b','c'],                   
                  'b': ['d','e'],                          
                  'c': ['f'],                     
                  'd': ['g', 'h'],                 
                  'f': ['i','j']}                   
def actionsF(s):                               
    try:                                      
        ## step cost of each action is 1
        return [(succ,1) for succ in successors[s]]
    except KeyError:
        return []
def takeActionF(s,a):
    return a
def goalTestF(s):
    return s == goal
def h1(s):
    return 0
start = 'a'
goal = 'f'
#result = aStarSearch(start,actionsF,takeActionF,goalTestF,h1)
#print('Path from a to h is', result[0], 'for a cost of', result[1])



In [5]:
def iterativeDeepeningSearch(startState, goalState, actionsF, takeActionF, maxDepth):
    path= 'cutoff'
    p=[]
    resultFound='false'
    for x in range(maxDepth+1):
        p= depthLimitedSearch(startState, goalState, actionsF, takeActionF, x)
        
        if p!='cutoff':
            return p
        
    if p=='cutoff':
        return 'cutoff'
    if p=='failure':
        return 'failure'
                


        

In [6]:
def depthLimitedSearch(startState , goalState, actionsF, takeActionF,depthLimit):
    expanded={}
    unexpanded= [(startState,"None")]
    solutionPath=[]
    existingChildren=[]
    depth=0
    needTuple='false'
    end='true'
    #If startState is the goalState, return the list containing just startState
    if startState==goalState:
        return startState
    else:
        while unexpanded:
            childrenAll=[]

            #Pop from the end of unExpanded a (state, parent) pair.
            (state,parent)= unexpanded.pop()
            if(state not in existingChildren):
                existingChildren.append(state);
            if(parent not in existingChildren):
                existingChildren.append(parent);

            #Add the node to the expanded dictionary, indexed by its state.
            if isinstance(state, list):
                state= tuple(state)
                needTuple= 'true'     
                
            expanded[state]=parent
            
            #check if depth limit has been reached
            searchParent=parent
            predecesor='true'
            steps=0
            while predecesor=='true':
                if searchParent in expanded.keys():
                    steps=steps+1
                    searchParent=expanded[searchParent]              
                else:
                    predecesor='false'
            children=[]
            
            
            if steps < depthLimit:
                #generates posible actions
                listActions=actionsF(state)
                #Generates successor depending on the posible moves  
                for eachAction in listActions:
                    c,cost=takeActionF(state, eachAction)
                    #c=takeActionF(state,eachAction)
                    if c not in existingChildren:
                        children.append(c)               
                #reverse children list 
                children=list(reversed(children))
                #insert children at the end of unexpanded list
                for p in children:
                    #state=list(state)
                    unexpanded.append((p,state))
            else:
                 #generates posible actions
                listActions=actionsF(state)
                #Generates successor depending on the posible moves  
                for eachAction in listActions:
                    c,cost=takeActionF(state, eachAction)
                    if c not in existingChildren:
                        children.append(c) 
                if len(children) > 0:
                    end='false'
                children=[]     
                
        

     
            #If the goal has been found (in python, goalState is in children):
            if goalState in children:  
                solutionPath.append(goalState)
                solutionPath.append(state)
                while parent in expanded.values():
                    if parent=="None":
                        break
                    solutionPath.append(parent)
                    parent=expanded[parent]
                if  needTuple=='true':
                    newSolutionPath=[]
                    for x in solutionPath:
                        lst= list(x)
                        newSolutionPath.append(lst)               
                    solutionPath= newSolutionPath 
                return list(reversed(solutionPath))
            
         
            
        if  end=='true':
            return "failure"
        else:
            return "cutoff"
        #return "cutoff"
        

In [7]:
def h1_8p(state, goal):
    return 0

In [8]:
import copy
def actionsF_simple(state):
    return  copy.copy(succs.get(state, []))

In [9]:
#dictionary of successors
successors = {'a': ['b', 'c'], 'b':['e', 'f', 'g'], 'b':['a'], 'c':['h'], 'h':['i'], 'i':['j', 'k', 'l'], 'k':['z']}

successors

{'a': ['b', 'c'],
 'b': ['a'],
 'c': ['h'],
 'h': ['i'],
 'i': ['j', 'k', 'l'],
 'k': ['z']}

In [10]:
#return valid actions paired with the single step cost
def actionsF_simple(s):                              #       \ 
        try:                                      #        j
            ## step cost of each action is 1
            return [(succ,1) for succ in successors[s]]
        except KeyError:
            return []    
actionsF_simple('a')

[('b', 1), ('c', 1)]

In [11]:
#return the pair containing the new state and the path cost so far from the start state to the new state 
def takeActionF_simple(s,a):
        return a
takeActionF_simple('a',('b', 1))

('b', 1)

In [12]:

def goalTestF_simple(s,goal):
    return s == goal


In [66]:
def h1_simple(s):
    return 0

In [14]:
iterativeDeepeningSearch('a', 'z', actionsF_simple, takeActionF_simple, 10)

['a', 'c', 'h', 'i', 'k', 'z']

# 8-Puzzle

In [15]:
def printPath_8p(startState):
    length= len(startState)
    dim=0
    for c in range(length):
        counter=c**2
        if counter==length: 
            dim=c              
    for i in range(dim):
        for j in range(dim):
             print(startState[i*dim+j]), 
        print ''


In [16]:
def findBlank_8p(startState):
    blankPosition=[]
    length= len(startState)
    dim= 0
    for c in range(length):
        counter=c**2
        if counter==length: 
            dim=c 
    if 0 in startState:
       
        for row in range(dim):          
             for column in range(dim): 
                    if startState[row*dim+column]==0:
                        blankPosition.append(row)
                        blankPosition.append(column)
                        
    return blankPosition



In [17]:
def actionsF_8p(startState):
   
    action=[]
    
    #gets puzzle demension
    dim=0
    length= len(startState)
    for c in range(length):
        counter=c**2
        if counter==length: 
            dim=c  
    
    blankPosition=findBlank_8p(startState)
    row=blankPosition[0]
    column=blankPosition[1]
    blankPosition=row*dim+column
   
    #check if there is blank position
    if blankPosition !=-1:
        action=['left', 'right', 'up', 'down']
      

        #check left border
        for row in range(dim):
                if blankPosition in [row*dim]:
                        action.remove('left')
                        
        #check right border
        for row in range(dim):
            if blankPosition in [(row*dim)+(dim-1)]:
                     action.remove('right')
        
        #check top border
        if blankPosition -dim <0:
            action.remove('up')
            
        #check bottom border
        if blankPosition +dim >=len(startState):
            action.remove("down")        
    
    actionCost=[]
    for ac in action:
        actionCost.append((ac,1))
    return actionCost

In [18]:
startState=[1, 0, 3, 5, 4, 2, 6, 7, 8]
if isinstance(startState, list):
    print 'c'

#actionsF_8p(startState)

c


In [19]:
def takeActionF_8p(startState, act):

    newState=[]
    invalidLeft='false'
    invalidRight='false'
    invalidDown='false'
    invalidUp='false'
    #gets puzzle demension
    dim=0
    length= len(startState)
    for c in range(length):
        counter=c**2
        if counter==length: 
            dim=c  
    
    blankPosition=findBlank_8p(startState)
    row=blankPosition[0]
    column=blankPosition[1]
    blankPosition=row*dim+column
    
    
    #check left border
    for r in range(dim):
        if blankPosition in [r*dim]:
            invalidLeft='true'
                   

    #check right border
    for row in range(dim):
        if blankPosition in [(r*dim)+(dim-1)]:
            invalidRight='true'

    #check top border
    if blankPosition -dim <0:
         invalidUp='true'

    #check bottom border
    if blankPosition +dim >=len(startState):
        invalidDown='true'  
    
    action=act[0]
    pos=0
    for x in startState:
        newState.append(x)
    if action=='left' and invalidLeft=='false':
        pos=blankPosition
        pos=pos-1
        newState[blankPosition]=startState[pos]
        newState[pos]=0
    if action=='right'and invalidRight=='false':
        pos=blankPosition
        pos=pos+1
        newState[blankPosition]=startState[pos]
        newState[pos]=0
    if action=='up' and invalidUp=='false':
        pos=blankPosition
        pos=pos-dim
        newState[blankPosition]=startState[pos]
        newState[pos]=0
    if action=='down' and invalidDown=='false':
        pos=blankPosition
        pos=pos+dim      
        newState[blankPosition]=startState[pos]
        newState[pos]=0
      
    return (newState,act[1])


In [20]:

def goalTestF_8p(s,goal):
    return s == goal


In [21]:
startState=[1, 0, 3, 5, 4, 2, 6, 7, 8]
takeActionF_8p(startState, ('left', 1))

([0, 1, 3, 5, 4, 2, 6, 7, 8], 1)

In [22]:
#aStarSearch('a',actionsF_simple, takeActionF_simple,
 #           lambda s: goalTestF_simple(s, 'z'),
  #          lambda s: hF_simple(s, 'z'))

In [23]:
 iterativeDeepeningSearch([5, 2, 8, 0, 1, 4, 3, 7, 6], 
                                 [0, 2, 3, 1, 4,  6, 7, 5, 8],
                                 actionsF_8p, takeActionF_8p, 10)

'cutoff'

## Heuristic functions

In [24]:
#given an item and  a state, returns the position of that item in the puzzle
def findPosition(startState, item):
    position=[]
    length= len(startState)
    dim= 0
    for c in range(length):
        counter=c**2
        if counter==length: 
            dim=c 
    if item in startState:
       
        for row in range(dim):          
             for column in range(dim): 
                    if startState[row*dim+column]==item:
                        position.append(row)
                        position.append(column)
                        
    return position

#findPosition([1, 2, 4, 4, 5, 6, 7, 0, 8], 3)


In [103]:
def h2_8p(startState, goalState):
    manhattanDistance=0
    
    statePosRow=0
    statePosCol=0
    goalPosRow=0
    goalPosCol=0
    
    stateItem=0
    goalItem=0
    
    length= len(startState)
    dim= 0
    for c in range(length):
        counter=c**2
        if counter==length: 
            dim=c 
    for row in range(dim):          
        for column in range(dim):
            stateItem=startState[row*dim+column]
            goalItem=goalState[row*dim+column]
            if stateItem!=goalItem and stateItem==0:
                statePosRow=row
                statePosCol=column
                goalPosRow,goalPosCol=findPosition(goalState,stateItem)
                x=abs(statePosRow-goalPosRow)+abs(statePosCol-goalPosCol)
                manhattanDistance+=x
            
    return manhattanDistance

#h2([7,2,4,5,0,6,8,3,1], [0,1, 2, 3, 4, 5, 6, 7, 8] )
            

In [37]:
aStarSearch([1, 2, 3, 4, 0, 5, 6, 7, 8],actionsF_8p, takeActionF_8p,
                     lambda s: goalTestF_8p(s, [1, 2, 3, 4, 5, 8, 6, 0, 7]),
                     lambda s: h2(s,[1, 2, 3, 4, 0, 5, 6, 7, 8]))



2
0


([[1, 2, 3, 4, 0, 5, 6, 7, 8],
  [1, 2, 3, 4, 5, 0, 6, 7, 8],
  [1, 2, 3, 4, 5, 8, 6, 7, 0],
  [1, 2, 3, 4, 5, 8, 6, 0, 7]],
 3)

In [97]:
def h3_8p(startState,goalState):
    misplacedTiles=0
 
    length= len(startState)
    dim= 0
    for c in range(length):
        counter=c**2
        if counter==length: 
            dim=c 
    for row in range(dim):          
        for column in range(dim):
            if startState[row*dim+column]!=goalState[row*dim+column]:
                misplacedTiles+=1

            
    return misplacedTiles

h3([7,2,4,5,0,6,8,3,1], [3,2,4,5,6,0,8,7,1])
            

4

## Run Experiment

In [136]:
 import pandas
def runExperiment(goalState1, goalState2, goalState3, heuristicFunctions):
   
    for hF in heuristicFunctions:
            aStarSearch([1, 2, 3, 4, 0, 5, 6, 7, 8],actionsF_8p, takeActionF_8p,
            lambda s: goalTestF_8p(s, goalState2),
            lambda s: hF(s,goalState2))

            data=[hF.__name__, depth, nNodes, ebf(nNodes,depth)]
            print data
            
            '''print  aStarSearch([1, 2, 3, 4, 0, 5, 6, 7, 8],actionsF_8p, takeActionF_8p,
                 lambda s: goalTestF_8p(s, goalState2),
            lambda s: hF(s,goalState2))


            print  aStarSearch([1, 2, 3, 4, 0, 5, 6, 7, 8],actionsF_8p, takeActionF_8p,
                 lambda s: goalTestF_8p(s, goalState3),
            lambda s: hF(s,goalState3))'''
        

runExperiment([1, 2, 3, 4, 0, 5, 6, 7, 8], [1, 2, 3, 4, 5, 8, 6, 0, 7], [1, 0, 3, 4, 5, 8, 2, 6, 7] , [h1_8p, h2_8p, h3_8p])

    
    

ImportError: No module named pandas

In [29]:
# from A3mysolution import *

In [30]:
%run -i A3grader.py

CRITICAL ERROR: Function named 'runExperiment' is not defined
  Check the spelling and capitalization of the function name.

Testing actionsF_8p([1, 2, 3, 4, 5, 6, 7, 0, 8])
('\n--- 5/5 points. Your actionsF_8p correctly returned', [('left', 1), ('right', 1), ('up', 1)])

Testing takeActionF_8p([1, 2, 3, 4, 5, 6, 7, 0, 8], (up, 1))
('\n--- 5/5 points. Your takeActionsF_8p correctly returned', ([1, 2, 3, 4, 0, 6, 7, 5, 8], 1))

Testing goalTestF_8p([1, 2, 3, 4, 5, 6, 7, 0, 8], [1, 2, 3, 4, 5, 6, 7, 0, 8])

--- 5/5 points. Your goalTestF_8p correctly True

Testing aStarSearch([1, 2, 3, 4, 5, 6, 7, 0, 8],
                     actionsF_8p, takeActionF_8p,
                     lambda s: goalTestF_8p(s, [0, 2, 3, 1, 4,  6, 7, 5, 8]),
                     lambda s: h1_8p(s, [0, 2, 3, 1, 4,  6, 7, 5, 8]))
('\n--- 20/20 points. Your search correctly returned', ([[1, 2, 3, 4, 5, 6, 7, 0, 8], [1, 2, 3, 4, 0, 6, 7, 5, 8], [1, 2, 3, 0, 4, 6, 7, 5, 8], [0, 2, 3, 1, 4, 6, 7, 5, 8]], 3))

Testing iter